In [10]:
import openmesh as om
import networkx as nx
import numpy

def find_middles(t):
    return (t[0] + t[1])/2, (t[1] + t[2])/2, (t[2] + t[0])/2

def has_vertex(mesh_t,v):
    for i, point in enumerate(mesh_t.points()):
            if all(point == mesh_t.point(v)) and v.idx() != i: return i
    return v.idx()


def triangulate_mesh (mesh, mesh_t):
    tri_dict = {}
    for fh in mesh.faces(): # triangulate each face
        triangle = []
        [triangle.append(mesh.point(vertex)) for vertex in mesh.fv(fh)] # populate the triangle with points of vertices
        a,b,c = [mesh_t.add_vertex(point) for point in triangle] # add the corners of the face to the triangulated mesh 
        d = mesh_t.add_vertex(numpy.mean(triangle,0)) # add the center point (mean of three points) to the triangulated mesh
        e,f,g = [mesh_t.add_vertex(midpoint) for midpoint in find_middles(triangle)] # calculatet he midpoints
        a,b,c,d,e,f,g = [mesh_t.vertex_handle(has_vertex(mesh_t,i)) for i in [a,b,c,d,e,f,g]] # check for duplicate vertices, if found, replace with original
        f1 = mesh_t.add_face(e,d,a) #create the six faces--vertices have to be in cw order
        f2 = mesh_t.add_face(b,d,e)
        f3 = mesh_t.add_face(f,d,b)
        f4 = mesh_t.add_face(c,d,f)
        f5 = mesh_t.add_face(g,d,c)
        f6 = mesh_t.add_face(a,d,g)
        mesh_t.delete_isolated_vertices() # remove vertices that used to be duplicates
        mesh_t.garbage_collection() # clean up the mesh
        tri_dict[fh.idx()] = [f1.idx(),f2.idx(),f3.idx(),f4.idx(),f5.idx(),f6.idx()]
    return mesh_t, tri_dict

def graph_from_mesh(m):
    G = nx.Graph()
    [G.add_node(i, triangle=i) for i, face in enumerate(m.faces())] # populate the graph with nodes (faces)      
    for node in G.nodes.data():
        for triangle in m.ff(m.face_handle(node[1]['triangle'])):      # find the adjacent triangles for every triangle
            c = [x for x,y in G.nodes(data=True) if (y['triangle']==triangle.idx())] # find the corresponding node for adjacent faces
            G.add_edge(node[0],c[0]) # add an edge to the graph between the current face and the adjacent face
    return G

mesh = om.TriMesh()
mesh = om.read_trimesh("house.stl")
mesh_t = om.TriMesh()
mesh_t, tri_dict = triangulate_mesh(mesh,mesh_t)
T = nx.minimum_spanning_tree(graph_from_mesh(mesh)) # find the spanning tree

print(T)
#print(mesh_t.face_vertex_indices())

#print(T.nodes.data()[0]['triangle'])
#print(tri_dict[T.nodes.data()[0]['triangle']])

#[print(T.nodes.data()[i]['triangle']) for i in range(12)]
print(T.edges())

#for treeNode in T.nodes():
#    print (treeNode.data()['triangle'])
triangle_list = []
i=0
node = T.nodes()[0]['triangle']
print (node)
debug = False
def next_triangle (adj_tris, next_triangles, curr_triangle):
    for tri in adj_tris:
        if debug:print ("flattened list of next_triangles :", [item for sublist in next_triangles for item in sublist])
        if (tri in [item for sublist in next_triangles for item in sublist]):
            return tri
    if debug:print ("do not go on a different triangle")
    return curr_triangle
def triangle_in_node (t,nodes,tri_dict, curr_node):
    if debug:print(tri_dict)
    if debug:print("t: ",t)
    for node in nodes:
        if debug:print ("checking node: ", node)
        if debug:print("tri-dict[node] : ", tri_dict[node])
        if t in tri_dict[node]:
            if debug:print("should continue with this node : ", node)
            return node    
    if debug:print("stay with current node : ", curr_node)
    return curr_node

def traverse(mesh_t, mesh, T, node, tri_dict, curr_triangle, triangle_list, c):
    print("=============================")
    if (mesh_t.face_property("visited", mesh_t.face_handle(curr_triangle)) == True):
        return triangle_list
    #if (c > 5):return
    print("traverse with curr_triangle: ", curr_triangle)
    # get the triangulated triangles from the current node
    triangles = tri_dict[node]
    if debug:print("traverse with triangles: ", triangles)
    # find the next nodes in the spanning tree
    next_nodes = nx.descendants_at_distance(T,node,1)
    if debug:print ("current node: ", node)
    if debug:print("found these next nodes: ", next_nodes)
    # find what triangles belong to those nodes
    next_triangles = [tri_dict[i] for i in next_nodes]
    if debug:print("found these next triangles: ", next_triangles)
    # add the triangle to the list 
    triangle_list.append(curr_triangle)
    # find what triangles are adjacent to the current one
    adj_tris = [f.idx() for f in mesh_t.ff(mesh_t.face_handle(curr_triangle))]
    if debug:print("found these adjacent triangles: ", adj_tris)
    # move to the next triangle, either the next in line OR an adjacent triangle on another node
    next_t = next_triangle(adj_tris, next_triangles, curr_triangle)
    if (next_t == curr_triangle or mesh_t.face_property("visited", mesh_t.face_handle(next_t)) == True):
        for tri in adj_tris:
            if (mesh_t.face_property("visited", mesh_t.face_handle(tri)) != True and tri in triangles): # check if the triangle is unvisted and on the current node
                if debug:print ("tri: ", tri)
                next_t = tri
                if debug:print ("found better solution")
                break
    if debug:print("will continue with : ", next_t)
    # remove the current triangle from the mesh_t
    mesh_t.set_face_property("visited", mesh_t.face_handle(curr_triangle), True)
    #mesh_t.delete_face(mesh_t.face_handle(curr_triangle))
    #mesh_t.garbage_collection()
    #recursively call the traversal method
    c+=1
    return traverse(mesh_t, mesh, T, triangle_in_node(next_t,next_nodes, tri_dict, node), tri_dict, next_t, triangle_list, c)
    # looks like triangle in node fails

print(traverse(mesh_t, mesh, T, 0, tri_dict, 0, triangle_list, 0))
#triangles = tri_dict[node]
#next_nodes = nx.descendants_at_distance(T,node,1)
#next_triangles = [tri_dict[i] for i in next_nodes]
#curr_triangle = triangles[0]
#triangle_list.append(curr_triangle)
#adj_tris = [f.idx() for f in mesh_t.ff(mesh_t.face_handle(curr_triangle))]
#for tri in adj_tris:
#    if (tri in [item for sublist in next_triangles for item in sublist]):
#        print (tri)
#    else: 
#        print("no neighbor")
#mesh_t.delete_face(mesh_t.face_handle(curr_triangle))

print()
# get all neighboring triangles
 #nextNodes have a triangle connected to this one

#for i in range(12):
#    print(i)
#    print("mesh:", [mesh.point(mesh.vertex_handle(v)) for v in mesh.face_vertex_indices()[i]])
#    print("mesh_t:", [mesh_t.point(mesh_t.vertex_handle(v)) for v in mesh_t.face_vertex_indices()[i+0]])
#    print("mesh_t:", [mesh_t.point(mesh_t.vertex_handle(v)) for v in mesh_t.face_vertex_indices()[i+1]])
#    print("mesh_t:", [mesh_t.point(mesh_t.vertex_handle(v)) for v in mesh_t.face_vertex_indices()[i+2]])
#    print("mesh_t:", [mesh_t.point(mesh_t.vertex_handle(v)) for v in mesh_t.face_vertex_indices()[i+3]])
#    print("mesh_t:", [mesh_t.point(mesh_t.vertex_handle(v)) for v in mesh_t.face_vertex_indices()[i+4]])
#    print("mesh_t:", [mesh_t.point(mesh_t.vertex_handle(v)) for v in mesh_t.face_vertex_indices()[i+5]])

Graph with 16 nodes and 15 edges
[(0, 1), (0, 4), (0, 13), (1, 15), (1, 7), (2, 4), (2, 14), (2, 11), (3, 4), (3, 6), (3, 12), (5, 6), (5, 8), (6, 10), (8, 9)]
0
traverse with curr_triangle:  0
traverse with curr_triangle:  27
traverse with curr_triangle:  28
traverse with curr_triangle:  17
traverse with curr_triangle:  12
traverse with curr_triangle:  87
traverse with curr_triangle:  88
traverse with curr_triangle:  89
traverse with curr_triangle:  84
traverse with curr_triangle:  85
traverse with curr_triangle:  86
traverse with curr_triangle:  13
traverse with curr_triangle:  14
traverse with curr_triangle:  69
traverse with curr_triangle:  70
traverse with curr_triangle:  71
traverse with curr_triangle:  66
traverse with curr_triangle:  67
traverse with curr_triangle:  68
traverse with curr_triangle:  15
traverse with curr_triangle:  16
traverse with curr_triangle:  29
traverse with curr_triangle:  24
traverse with curr_triangle:  23
traverse with curr_triangle:  18
traverse with 

In [ ]:
TODO NEXT: make sure it traverses every node of 